## Deliverable 3. Create a Travel Itinerary Map.

In [38]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import gmaps.datasets

# Import API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

In [39]:
# 1. Read the WeatherPy_vacation.csv into a DataFrame.
vacation_df = pd.read_csv("../Vacation_Search/WeatherPy_vacation.csv")
vacation_df.head()

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,0,Hilo,US,67.73,overcast clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
1,4,Georgetown,MY,83.89,few clouds,5.4112,100.3354,Cititel Penang
2,5,Lorengau,PG,80.80,overcast clouds,-2.0226,147.2712,Lorengau Harbourside Hotel
3,7,Port Alfred,ZA,80.51,clear sky,-33.5906,26.8910,The Halyards Hotel
4,9,Naze,JP,62.06,overcast clouds,28.3667,129.4833,Shishime Hotel


In [40]:
# 2. Using the template add the city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 3a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in vacation_df.iterrows()]

# 3b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = vacation_df[["Lat", "Lng"]]

In [41]:
# 4a. Add a marker layer for each city to the map.
locations = vacation_df[["Lat", "Lng"]]
max_temp = vacation_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                 weights=[max(temp, 0) for temp in max_temp],
                dissipating=False,
                 max_intensity=300, 
                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 4b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))

In [42]:
# From the map above pick 4 cities and create a vacation itinerary route to travel between the four cities. 
# 5. Create DataFrames for each city by filtering the 'vacation_df' using the loc method. 
# Hint: The starting and ending city should be the same city.

vacation_start = vacation_df.loc[vacation_df['City'] == 'Villarrobledo']
vacation_end = vacation_df.loc[vacation_df['City'] == 'Villarrobledo']
vacation_stop1 = vacation_df.loc[vacation_df['City'] == 'San Pedro Del Pinatar']
vacation_stop2 = vacation_df.loc[vacation_df['City'] == 'Cabra'] 
vacation_stop3 = vacation_df.loc[vacation_df['City'] == 'Barbate'] 

In [43]:
vacation_start

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
322,642,Villarrobledo,ES,60.87,broken clouds,39.2699,-2.6012,Hotel Castillo


In [44]:
vacation_end

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
322,642,Villarrobledo,ES,60.87,broken clouds,39.2699,-2.6012,Hotel Castillo


In [45]:
vacation_stop1

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
47,87,San Pedro Del Pinatar,ES,64.83,few clouds,37.8357,-0.791,Thalasia Costa de Murcia


In [46]:
vacation_stop2

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
198,392,Cabra,ES,65.14,broken clouds,37.4725,-4.4421,MS Fuente Las Piedras


In [47]:
vacation_stop3

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
9,18,Barbate,ES,68.09,overcast clouds,36.1924,-5.9219,Hotel Adiafa


In [48]:
# 6. Get the latitude-longitude pairs as tuples from each city DataFrame using the to_numpy function and list indexing.
start = vacation_start["Lat"].to_numpy()[0], vacation_start["Lng"].to_numpy()[0]
end = vacation_end["Lat"].to_numpy()[0], vacation_end["Lng"].to_numpy()[0]
stop1 = vacation_stop1["Lat"].to_numpy()[0], vacation_stop1["Lng"].to_numpy()[0]
stop2 = vacation_stop2["Lat"].to_numpy()[0], vacation_stop2["Lng"].to_numpy()[0]
stop3 = vacation_stop3["Lat"].to_numpy()[0], vacation_stop3["Lng"].to_numpy()[0]

In [49]:
start

(39.2699, -2.6012)

In [50]:
# 7. Create a direction layer map using the start and end latitude-longitude pairs,
# and stop1, stop2, and stop3 as the waypoints. The travel_mode should be "DRIVING", "BICYCLING", or "WALKING".

import gmaps
import gmaps.datasets

fig = gmaps.figure()
directions_graph = gmaps.directions_layer(start, end, waypoints=[stop1, stop2, stop3], travel_mode = 'DRIVING')

fig.add_layer(directions_graph)
fig


Figure(layout=FigureLayout(height='420px'))

In [51]:
# 8. To create a marker layer map between the four cities.
#  Combine the four city DataFrames into one DataFrame using the concat() function.
itinerary_df = pd.concat([vacation_start, vacation_stop1, vacation_stop2, vacation_stop3, vacation_end], ignore_index=True)
itinerary_df

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,642,Villarrobledo,ES,60.87,broken clouds,39.2699,-2.6012,Hotel Castillo
1,87,San Pedro Del Pinatar,ES,64.83,few clouds,37.8357,-0.7910,Thalasia Costa de Murcia
2,392,Cabra,ES,65.14,broken clouds,37.4725,-4.4421,MS Fuente Las Piedras
3,18,Barbate,ES,68.09,overcast clouds,36.1924,-5.9219,Hotel Adiafa
4,642,Villarrobledo,ES,60.87,broken clouds,39.2699,-2.6012,Hotel Castillo


In [52]:
# 9 Using the template add city name, the country code, the weather description and maximum temperature for the city. 
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in itinerary_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = itinerary_df[["Lat", "Lng"]]

In [53]:
# 11a. Add a marker layer for each city to the map.
locations = itinerary_df[["Lat", "Lng"]]
max_temp = itinerary_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, 
                 weights=[max(temp, 0) for temp in max_temp],
                 dissipating=False,
                 max_intensity=300, 
                 point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))